In [1]:
import pandas as pd
from IPython.display import display
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier, plot_tree, export_text
from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score
import matplotlib.pyplot as plt

In [2]:
pd.reset_option('display.max_rows')
# Imposta pandas per mostrare tutte le colonne senza abbreviazioni
pd.set_option('display.max_columns', None)

In [3]:
file_path = 'Export\DataFrame\df_cv_job_PreProc.pkl'

# Carica il file pickle in un DataFrame
df_cv_job = pd.read_pickle(file_path)


df_cv_job.head()

,kId_cv,qId_cv,age_cv,competences_coverage_required,competences_coverage_optional,knowledge_coverage_required,knowledge_coverage_optional,copertura_lingue_obbligatorie,copertura_lingue_opzionali,years_experience_cv,year_lastjob_cv,years_experience_job,years_experience_job_req,education_cv_encoded,educations_job_req,educations_job_encoded,isConsistent,contract_job_Hybrid,contract_job_In Presence,contract_job_Remotely,contract_cv_Any,contract_cv_Hybrid,contract_cv_In Presence,contract_cv_Remotely,gender_job_Any,gender_job_Female,gender_job_Male,gender_cv_Female,gender_cv_Male,gender_cv_Not binary,gender_cv_Not declared,disability_cv_No,disability_cv_Not declared,disability_cv_Yes,disability_job_Any,disability_job_No,PCA_1,PCA_2,nationality_job_Any,nationality_job_Catalan,nationality_job_English,nationality_job_French,nationality_job_Italian,nationality_job_Spanish,nationality_cv_Catalan,nationality_cv_English,nationality_cv_French,nationality_cv_German,nationality_cv_Irish,nationality_cv_Italian,nationality_cv_LowGerman,nationality_cv_Romanian,nationality_cv_Russian,nationality_cv_Spanish,nationality_cv_Swedish,city_cv_encoded,age_min_job,age_max_job
0,0,0,21,0.50,0.0,0.75,0.0,0.0,1.0,0,-1,2,1,0,1,3,0,False,True,False,False,False,True,False,False,False,True,False,True,False,False,False,False,True,False,True,-0.009232,-0.012665,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,0.345824,25,33
1,1,0,50,0.50,0.0,1.00,0.0,0.0,1.0,2,-1,2,1,2,1,3,0,False,True,False,False,False,True,False,False,False,True,True,False,False,False,False,False,True,False,True,-0.000368,-0.004032,True,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,0.456838,25,33
2,2,0,23,0.75,0.0,0.25,0.0,1.0,0.0,4,-1,2,1,1,1,3,0,False,True,False,False,False,True,False,False,False,True,True,False,False,False,False,False,True,False,True,-0.016230,0.000786,True,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,0.389489,25,33
3,3,0,47,0.75,0.0,0.75,0.0,0.0,1.0,2,2018,2,1,2,1,3,0,False,True,False,False,False,True,False,False,False,True,True,False,False,False,False,True,False,False,True,0.010315,-0.003472,True,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,0.362250,25,33
4,4,0,20,0.75,0.0,1.00,0.0,0.0,1.0,5,-1,2,1,0,1,3,0,False,True,False,False,False,True,False,False,False,True,False,True,False,False,True,False,False,False,True,0.005693,-0.002401,True,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,0.460090,25,33


In [4]:
df_cv_job=df_cv_job.drop(columns=['kId_cv'])

In [5]:
df_cv_job.rename(columns={'educations_job_req': 'education_job_req'}, inplace=True)
df_cv_job.rename(columns={'educations_job_encoded': 'education_job_encoded'}, inplace=True)


## creiamo test e training set facendo in modo di avere i valori di qId uguali nello stesso set

In [6]:
from sklearn.model_selection import GroupShuffleSplit

In [7]:
# Definire le feature e la variabile target
X = df_cv_job.drop(columns=['isConsistent'])  # Elimina la colonna target
y = df_cv_job['isConsistent']  # Colonna target

# Creare il GroupShuffleSplit per garantire che i valori di qId_cv non siano divisi tra training e test
gss = GroupShuffleSplit(test_size=0.2, n_splits=1, random_state=54)
train_idx, test_idx = next(gss.split(X, groups=df_cv_job['qId_cv']))

# Creare il training e il test set
X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

In [8]:
X_train['qId_cv'].value_counts()

qId_cv
0     500
1     500
23    500
25    500
26    500
27    500
29    500
30    500
31    500
32    500
33    500
35    500
36    500
41    500
42    500
44    500
45    500
46    500
47    500
22    500
21    500
20    500
9     500
2     500
3     500
4     500
5     500
6     500
7     500
8     500
11    500
19    500
12    500
13    500
14    500
15    500
16    500
17    500
18    500
48    500
Name: count, dtype: int64

In [9]:
X_test['qId_cv'].value_counts()

qId_cv
10    500
24    500
28    500
34    500
37    500
38    500
39    500
40    500
43    500
49    500
Name: count, dtype: int64

## Decision Tree

In [10]:

clf = DecisionTreeClassifier(random_state=1)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
baseline_accuracy = accuracy_score(y_test, y_pred)
baseline_precision=precision_score(y_test,y_pred)
baseline_recall=recall_score(y_test,y_pred)
report = classification_report(y_test, y_pred)


print(f"Accuracy: {baseline_accuracy:.4f}")
print(f"Precision: {baseline_precision:.4f}")
print(f"Recall: {baseline_recall:.4f}")
print("Classification Report:")
print(report)

Accuracy: 0.8504
Precision: 0.8937
Recall: 0.7105
Classification Report:
              precision    recall  f1-score   support

           0       0.83      0.94      0.88      3000
           1       0.89      0.71      0.79      2000

    accuracy                           0.85      5000
   macro avg       0.86      0.83      0.84      5000
weighted avg       0.86      0.85      0.85      5000



In [11]:
from sklearn.metrics import roc_auc_score
y_pred_proba = clf.predict_proba(X_test)[:, 1]
baseline_roc_auc = roc_auc_score(y_test, y_pred_proba)
print(f"ROC AUC: {baseline_roc_auc:.4f}")

ROC AUC: 0.8271


In [12]:
"""

param_grid = {
    'criterion': ['gini', 'entropy'],  # Funzione di split
    'splitter': ['best', 'random'],    # Strategia per scegliere la feature
    'max_depth': [None, 30, 40, 50],   # Profondità massima dell'albero
    'min_samples_split': [20, 30, 40],  # Numero minimo di campioni richiesti per uno split
    'min_samples_leaf': [1, 5, 10],    # Numero minimo di campioni in un nodo foglia
    ''
}

clf = DecisionTreeClassifier(random_state=46)

grid_search = GridSearchCV(clf, param_grid, cv=5, scoring='accuracy', n_jobs=-1)

grid_search.fit(X_train_resampled, y_train_resampled)

best_params = grid_search.best_params_

print("Best hyperparameters:", best_params)

best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print(f"Accuracy: {accuracy:.4f}")
print("Classification Report:")
print(report)
"""

'\n\nparam_grid = {\n    \'criterion\': [\'gini\', \'entropy\'],  # Funzione di split\n    \'splitter\': [\'best\', \'random\'],    # Strategia per scegliere la feature\n    \'max_depth\': [None, 30, 40, 50],   # Profondità massima dell\'albero\n    \'min_samples_split\': [20, 30, 40],  # Numero minimo di campioni richiesti per uno split\n    \'min_samples_leaf\': [1, 5, 10],    # Numero minimo di campioni in un nodo foglia\n    \'\'\n}\n\nclf = DecisionTreeClassifier(random_state=46)\n\ngrid_search = GridSearchCV(clf, param_grid, cv=5, scoring=\'accuracy\', n_jobs=-1)\n\ngrid_search.fit(X_train_resampled, y_train_resampled)\n\nbest_params = grid_search.best_params_\n\nprint("Best hyperparameters:", best_params)\n\nbest_model = grid_search.best_estimator_\ny_pred = best_model.predict(X_test)\n\naccuracy = accuracy_score(y_test, y_pred)\nreport = classification_report(y_test, y_pred)\n\nprint(f"Accuracy: {accuracy:.4f}")\nprint("Classification Report:")\nprint(report)\n'

In [13]:
# Ottenere l'importanza delle caratteristiche dal modello addestrato
feature_importances = clf.feature_importances_

feature_importances_df = pd.DataFrame({
    'Feature': list(X.columns), 
    'Importance': feature_importances
})

feature_importances_df = feature_importances_df.sort_values(by='Importance', ascending=False)

feature_importances_df.head()

,Feature,Importance
1,age_cv,0.295632
24,gender_job_Male,0.127034
26,gender_cv_Male,0.078901
12,education_cv_encoded,0.066744
55,age_max_job,0.055883


## Raggruppiamo le feature importance

In [14]:
# Creare una nuova colonna 'group' che contiene la parte prima del '_'
feature_importances_df['group'] = feature_importances_df['Feature'].apply(lambda x: x.split('_')[0])

# Raggruppare per la colonna 'group' e sommare le feature importance
df_grouped = feature_importances_df.groupby('group')['Importance'].sum().reset_index()
print(df_grouped.sort_values(by='Importance', ascending=False))

          group  Importance
1           age    0.379860
8        gender    0.299070
7     education    0.097291
6    disability    0.042560
3   competences    0.040418
10  nationality    0.038425
0           PCA    0.032214
2          city    0.022092
9     knowledge    0.013657
13        years    0.011594
11          qId    0.008182
4      contract    0.006818
12         year    0.004685
5     copertura    0.003133


In [15]:
# Unire le righe con 'educations' e 'education' sotto 'education' e sommare le loro importanze
#df_grouped.loc[df_grouped['group'].isin(['educations', 'education']), 'group'] = 'education'

df_grouped = df_grouped.groupby('group', as_index=False).sum()

# Modifica del valore 'copertura' in 'languages'
#df_grouped['group'] = df_grouped['group'].replace('copertura', 'languages')

df_grouped=df_grouped.sort_values(by='Importance', ascending=True)
df_grouped

,group,Importance
5,copertura,0.003133
12,year,0.004685
4,contract,0.006818
11,qId,0.008182
13,years,0.011594
9,knowledge,0.013657
2,city,0.022092
0,PCA,0.032214
10,nationality,0.038425
3,competences,0.040418


## Per introdurre il concetto di minimizzazione dei dati, proviamo a riaddestrare il modello togliendo progressivamente le feature dalla meno importante alla più importante, terminando il processo quando la Recall del modello riaddestrato diventa inferiore del 5% rispetto alla Recall iniziale. è stata scelta come baseline la metrica Recall, poichè era la metrica con perfomance peggiori inizialmente, e quindi l'obiettivo è di non farla peggiorare troppo ulteriormente.

In [16]:
# Tolleranza per la perdita di recall
recall_tolerance = 0.1  # 5%

In [17]:
list(df_grouped['group'])

['copertura',
 'year',
 'contract',
 'qId',
 'years',
 'knowledge',
 'city',
 'PCA',
 'nationality',
 'competences',
 'disability',
 'education',
 'gender',
 'age']

In [18]:
currentElement = ""
previousElement= ""
X_train_red=pd.DataFrame(X_train)
X_test_red=pd.DataFrame(X_test)
for element in df_grouped['group']:
    currentElement=element
    columns_to_drop = [col for col in X_train_red.columns if col.split('_')[0] == currentElement]
    X_train_red=X_train_red.drop(columns=columns_to_drop, errors='ignore')
    X_test_red=X_test_red.drop(columns=columns_to_drop, errors='ignore')
    # Riaddestrare il modello
    clf = DecisionTreeClassifier(random_state=1)
    clf.fit(X_train_red, y_train)
    # Calcolare la nuova recall
    y_pred_reduced = clf.predict(X_test_red)
    current_recall = recall_score(y_test, y_pred_reduced)
    print(f"Nuova Recall: {current_recall:.4f}")
    if current_recall <= (baseline_recall * (1 - recall_tolerance)):
        print (f"Threshold di recall superato {current_recall}: è possibile utilizzare il modello togliendo le feature fino a : {previousElement}")
        break
    previousElement=currentElement
feature_to_remove=df_grouped['group'][:df_grouped['group'].tolist().index(previousElement) + 1].tolist()
feature_to_remove


    

Nuova Recall: 0.7400
Nuova Recall: 0.7385
Nuova Recall: 0.6945
Nuova Recall: 0.6860
Nuova Recall: 0.7135
Nuova Recall: 0.6880
Nuova Recall: 0.7095
Nuova Recall: 0.6245
Threshold di recall superato 0.6245: è possibile utilizzare il modello togliendo le feature fino a : city


['copertura', 'year', 'contract', 'qId', 'years', 'knowledge', 'city']

In [19]:
columns_to_drop = [col for col in pd.DataFrame(X_train).columns if col.split('_')[0] in feature_to_remove]
X_train_minimized=pd.DataFrame(X_train).drop(columns=columns_to_drop, errors='ignore')
X_test_minimized=pd.DataFrame(X_test).drop(columns=columns_to_drop, errors='ignore')
X_train_minimized

,age_cv,competences_coverage_required,competences_coverage_optional,education_cv_encoded,education_job_req,education_job_encoded,gender_job_Any,gender_job_Female,gender_job_Male,gender_cv_Female,gender_cv_Male,gender_cv_Not binary,gender_cv_Not declared,disability_cv_No,disability_cv_Not declared,disability_cv_Yes,disability_job_Any,disability_job_No,PCA_1,PCA_2,nationality_job_Any,nationality_job_Catalan,nationality_job_English,nationality_job_French,nationality_job_Italian,nationality_job_Spanish,nationality_cv_Catalan,nationality_cv_English,nationality_cv_French,nationality_cv_German,nationality_cv_Irish,nationality_cv_Italian,nationality_cv_LowGerman,nationality_cv_Romanian,nationality_cv_Russian,nationality_cv_Spanish,nationality_cv_Swedish,age_min_job,age_max_job
0,21,0.500000,0.0,0,1,3,False,False,True,False,True,False,False,False,False,True,False,True,-0.009232,-0.012665,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,25,33
1,50,0.500000,0.0,2,1,3,False,False,True,True,False,False,False,False,False,True,False,True,-0.000368,-0.004032,True,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,25,33
2,23,0.750000,0.0,1,1,3,False,False,True,True,False,False,False,False,False,True,False,True,-0.016230,0.000786,True,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,25,33
3,47,0.750000,0.0,2,1,3,False,False,True,True,False,False,False,False,True,False,False,True,0.010315,-0.003472,True,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,25,33
4,20,0.750000,0.0,0,1,3,False,False,True,False,True,False,False,True,False,False,False,True,0.005693,-0.002401,True,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,25,33
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24495,33,1.000000,0.0,3,1,2,False,False,True,False,True,False,False,True,False,False,True,False,-0.015422,-0.008176,True,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,25,42
24496,40,0.666667,0.0,2,1,2,False,False,True,False,True,False,False,True,False,False,True,False,0.017542,0.004894,True,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,25,42
24497,33,0.666667,0.0,2,1,2,False,False,True,False,True,False,False,False,False,True,True,False,0.001231,-0.002544,True,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,25,42
24498,27,0.666667,0.0,2,1,2,False,False,True,False,True,False,False,True,False,False,True,False,0.003734,0.004856,True,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,25,42


RIADDESTRIAMO IL MODELLO CON LE FEATURE RIMASTE

In [20]:
clf = DecisionTreeClassifier(random_state=42)
clf.fit(X_train_minimized, y_train)
y_pred_min = clf.predict(X_test_minimized)
accuracy = accuracy_score(y_test, y_pred_min)
precision=precision_score(y_test,y_pred_min)
recall=recall_score(y_test,y_pred_min)
report = classification_report(y_test, y_pred_min)


print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print("Classification Report:")
print(report)

Accuracy: 0.8394
Precision: 0.9074
Recall: 0.6665
Classification Report:
              precision    recall  f1-score   support

           0       0.81      0.95      0.88      3000
           1       0.91      0.67      0.77      2000

    accuracy                           0.84      5000
   macro avg       0.86      0.81      0.82      5000
weighted avg       0.85      0.84      0.83      5000



RICALCOLIAMO LA FEATURE IMPORTANCE

In [21]:
# Ottenere l'importanza delle caratteristiche dal modello addestrato
feature_importances = clf.feature_importances_

feature_importances_df = pd.DataFrame({
    'Feature': list(X_train_minimized.columns), 
    'Importance': feature_importances
})

feature_importances_df = feature_importances_df.sort_values(by='Importance', ascending=False)

feature_importances_df.head()

,Feature,Importance
0,age_cv,0.300062
8,gender_job_Male,0.127380
10,gender_cv_Male,0.078901
3,education_cv_encoded,0.067325
38,age_max_job,0.062224


In [22]:
# Creare una nuova colonna 'group' che contiene la parte prima del '_'
feature_importances_df['group'] = feature_importances_df['Feature'].apply(lambda x: x.split('_')[0])

# Raggruppare per la colonna 'group' e sommare le feature importance
df_grouped = feature_importances_df.groupby('group')['Importance'].sum().reset_index()
print(df_grouped.sort_values(by='Importance', ascending=False))

         group  Importance
1          age    0.384539
5       gender    0.299539
4    education    0.122050
3   disability    0.056541
6  nationality    0.046635
0          PCA    0.045430
2  competences    0.045267


In [23]:
df_grouped=df_grouped.sort_values(by='Importance', ascending=False)
df_grouped

,group,Importance
1,age,0.384539
5,gender,0.299539
4,education,0.122050
3,disability,0.056541
6,nationality,0.046635
0,PCA,0.045430
2,competences,0.045267


Salviamo le variabili X_test y_pred e y_test per poterle usare in altro notebook per il calcolo delle metriche 

In [24]:
import pickle

In [25]:
# 1. Salva X_test in un file pickle
with open('X_test_MIN.pkl', 'wb') as file:
    pickle.dump(X_test_minimized, file)

# 2. Salva y_pred in un file pickle
with open('y_pred_MIN.pkl', 'wb') as file:
    pickle.dump(y_pred_min, file)

# 3. Salva y_test in un file pickle
with open('y_test.pkl', 'wb') as file:
    pickle.dump(y_test, file)